In [1]:
import pandas as pd
import os

files = [file for file in os.listdir('.') if file.endswith('csv')]
model_names = list(set([file.split('_')[-1].split('.')[0] for file in files]))
print('all models', model_names)


# slatewise ,'NeuralClickModelBest',
model_names = {
    'MatrixFactorization': 'Матричная факторизация', 
    'LogReg': 'Логистическая Регрессия', 
    'NeuralClickModel': 'Slate-wise Gated Recurrent Unit', 
    'NeuralClickModelBest': 'NeuralClickModel',
    'InSlateAttention' : 'Slate-wise Transformer'
}

# sequencewise
# model_names = [
#     'FlatAttentionClickedOnly', 'InSlateAttentionSequencewiseGRU', 'FlattenedGRU'
# ]

all models ['NeuralClickModelBest', 'MatrixFactorization', 'NeuralClickModelBase', 'FlatAttentionClickedOnly', 'FlatAttentionClickedOnly2LinearLnorm', 'NeuralClickModel', 'FlattenedGRU', 'InSlateAttentionSequencewiseGRU', 'LogReg', 'InSlateAttention']


In [2]:
data = {}

dataset_rename = {'cw': 'ContentWise', 'rl4rs': 'RL4RS'}
metrics_rename = {'f1':'F1', 'accuracy': 'Accuracy', 'roc-auc':"ROC-AUC"}
embeddings_rename = {'svd':'SVD', 'neural':"nn.Embeddings", 'explicit':"Explicit"}

index = pd.MultiIndex.from_product(
    [["ContentWise", "RL4RS"], ["ROC-AUC", "F1", "Accuracy"]]
)

columns = pd.MultiIndex.from_product(
    [model_names.values(), ["SVD", "nn.Embeddings", "Explicit"]]
)


df = pd.DataFrame(
    index=index,
    columns=columns,
)
data = df.to_dict()
data

{('Матричная факторизация', 'SVD'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('Матричная факторизация', 'nn.Embeddings'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('Матричная факторизация', 'Explicit'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('Логистическая Регрессия', 'SVD'): {('ContentWise', 'ROC-AUC'): nan,
  ('ContentWise', 'F1'): nan,
  ('ContentWise', 'Accuracy'): nan,
  ('RL4RS', 'ROC-AUC'): nan,
  ('RL4RS', 'F1'): nan,
  ('RL4RS', 'Accuracy'): nan},
 ('Логистическая Регрессия',
  'nn.Embeddings'): {('ContentWise', 'ROC-AUC'): nan, ('ContentWise',
   'F

In [3]:
for file in files:
    dataset = dataset_rename[file.split('_')[0]]
    try:
        model = model_names[file.split('_')[-1].split('.')[0]]
    except:
        print(file, 'is unknown model')
        continue
    data_dict = pd.read_csv(file).drop('Unnamed: 0', axis=1).set_index('embeddings').to_dict('index')
    for embedding, m in data_dict.items():
        for metric, score in m.items():
#             print(model, embeddings_rename[embedding], dataset, metrics_rename[metric], score )
            try:
                data[model, embeddings_rename[embedding]][dataset, metrics_rename[metric]] = score
            except:
                pass
        

cw_FlatAttentionClickedOnly2LinearLnorm.csv is unknown model
rl4rs_InSlateAttentionSequencewiseGRU.csv is unknown model
rl4rs_FlatAttentionClickedOnly2LinearLnorm.csv is unknown model
cw_InSlateAttentionSequencewiseGRU.csv is unknown model
rl4rs_NeuralClickModelBase.csv is unknown model
cw_NeuralClickModelBase.csv is unknown model
rl4rs_FlatAttentionClickedOnly.csv is unknown model
cw_FlatAttentionClickedOnly.csv is unknown model
rl4rs_FlattenedGRU.csv is unknown model
cw_FlattenedGRU.csv is unknown model


In [4]:
df = pd.DataFrame(
    data,
    index=index,
    columns=columns,
).T.dropna(how='all')
df

ContentWise                      \
                                                  ROC-AUC        F1  Accuracy   
Матричная факторизация          SVD              0.653530  0.198183  0.257367   
Логистическая Регрессия         SVD              0.689643  0.241118  0.842561   
                                nn.Embeddings    0.688562  0.264406  0.793215   
                                Explicit              NaN       NaN       NaN   
Slate-wise Gated Recurrent Unit SVD              0.722159  0.290024  0.867749   
                                nn.Embeddings    0.723409  0.293693  0.870500   
                                Explicit              NaN       NaN       NaN   
NeuralClickModel                SVD              0.722158  0.290024  0.867749   
                                nn.Embeddings    0.724738  0.282670  0.872236   
                                Explicit              NaN       NaN       NaN   
Slate-wise Transformer          SVD              0.749389  0.313946  0.874461   
                                nn.Embeddings    0.738976  0.323503  0.860084   
                                Explicit              NaN       NaN       NaN   

                                                  RL4RS                      
                                                ROC-AUC        F1  Accuracy  
Матричная факторизация          SVD            0.718950  0.755192  0.653391  
Логистическая Регрессия         SVD            0.912938  0.874831  0.836078  
                                nn.Embeddings  0.913241  0.874566  0.837359  
                                Explicit       0.913952  0.876435  0.837021  
Slate-wise Gated Recurrent Unit SVD            0.915465  0.874637  0.842317  
                                nn.Embeddings  0.911649  0.873242  0.839850  
                                Explicit       0.915695  0.877034  0.844662  
NeuralClickModel                SVD            0.917033  0.875962  0.843719  
                                nn.Embeddings  0.916946  0.877959  0.846524  
                                Explicit       0.909064  0.872293  0.839318  
Slate-wise Transformer          SVD            0.925636  0.884445  0.847540  
                                nn.Embeddings  0.927064  0.885775  0.848628  
                                Explicit       0.928539  0.885196  0.849498

In [5]:
df = df.style.format(decimal='.', precision=3).highlight_max(axis=0, props="font-weight:bold;")

In [6]:
df

In [7]:
print(df.to_latex(convert_css=True))

\begin{tabular}{llrrrrrr}
{} & {} & \multicolumn{3}{r}{ContentWise} & \multicolumn{3}{r}{RL4RS} \\
{} & {} & {ROC-AUC} & {F1} & {Accuracy} & {ROC-AUC} & {F1} & {Accuracy} \\
Матричная факторизация & SVD & 0.654 & 0.198 & 0.257 & 0.719 & 0.755 & 0.653 \\
\multirow[c]{3}{*}{Логистическая Регрессия} & SVD & 0.690 & 0.241 & 0.843 & 0.913 & 0.875 & 0.836 \\
 & nn.Embeddings & 0.689 & 0.264 & 0.793 & 0.913 & 0.875 & 0.837 \\
 & Explicit & nan & nan & nan & 0.914 & 0.876 & 0.837 \\
\multirow[c]{3}{*}{Slate-wise Gated Recurrent Unit} & SVD & 0.722 & 0.290 & 0.868 & 0.915 & 0.875 & 0.842 \\
 & nn.Embeddings & 0.723 & 0.294 & 0.870 & 0.912 & 0.873 & 0.840 \\
 & Explicit & nan & nan & nan & 0.916 & 0.877 & 0.845 \\
\multirow[c]{3}{*}{NeuralClickModel} & SVD & 0.722 & 0.290 & 0.868 & 0.917 & 0.876 & 0.844 \\
 & nn.Embeddings & 0.725 & 0.283 & 0.872 & 0.917 & 0.878 & 0.847 \\
 & Explicit & nan & nan & nan & 0.909 & 0.872 & 0.839 \\
\multirow[c]{3}{*}{Slate-wise Transformer} & SVD & \bfseries 0.749 